# SPSS Modelerのユーザー入力ノードをPythonで書き換える。

# 1．連番データ作成

In [1]:
import pandas as pd
pd.DataFrame(range(1,14),columns=['番号'])
#pd.DataFrame({'番号': pd.Series(range(1,14), dtype=int)})

,番号
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


Python | range関数の使い方(開始から終了までの連続した数値を要素として持つrange型オブジェクトを作成する) 

https://www.javadrive.jp/python/function/index6.html

# 2．デカルト積データ作成

In [2]:
import itertools
no=range(1,14)
card=['スペード','クラブ','ハート','ダイヤ']
pd.set_option('display.max_rows', 10)
pd.DataFrame(list(itertools.product(no,card)),columns=['番号','絵柄'])
#prod=list(itertools.product(no,card))
#pd.DataFrame({'番号': pd.Series([r[0] for r in prod], dtype=int),
#              '絵柄':pd.Series([r[1] for r in prod], dtype=object)})

,番号,絵柄
0,1,スペード
1,1,クラブ
2,1,ハート
3,1,ダイヤ
4,2,スペード
...,...,...
47,12,ダイヤ
48,13,スペード
49,13,クラブ
50,13,ハート


* 参考
Pythonで複数のリストの直積（デカルト積）を生成するitertools.product | note.nkmk.me 

https://note.nkmk.me/python-itertools-product/

# 3. 予測モデルのスコアリングデータを作成（1レコード）

In [3]:
#!wget 'https://raw.githubusercontent.com/yoichiro0903n/blue/main/cars.csv'
df=pd.read_csv('cars.csv')
df

,燃費,エンジン,馬力,車体重量,加速性,年式,生産国,気筒数
0,19.0,70,97,2330,13.5,72.0,日本,3気筒
1,18.0,70,90,2124,13.5,73.0,日本,3気筒
2,21.5,80,110,2720,13.5,77.0,日本,3気筒
3,23.7,70,100,2420,12.5,80.0,日本,3気筒
4,20.3,131,103,2830,15.9,78.0,ヨーロッパ,5気筒
...,...,...,...,...,...,...,...,...
387,22.0,146,97,2815,14.5,77.0,日本,6気筒
388,32.7,168,132,2910,11.4,80.0,日本,6気筒
389,25.4,168,116,2900,12.6,81.0,日本,6気筒
390,24.2,146,120,2930,13.8,81.0,日本,6気筒


In [4]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

X = df[['燃費', 'エンジン', '車体重量', '加速性']]         # 説明変数
Y = df['馬力']         # 目的変数

lr.fit(X, Y)                         # 線形モデルの重みを学習

LinearRegression()

In [5]:
df_scoring=pd.DataFrame([[21.9,86,2841,12.5]],
                        columns=['燃費', 'エンジン', '車体重量', '加速性'])
df_scoring

,燃費,エンジン,車体重量,加速性
0,21.9,86,2841,12.5


https://note.nkmk.me/python-pandas-dataframe-values-columns-index/

In [6]:
df_scoring['$-馬力']=lr.predict(df_scoring)
df_scoring

,燃費,エンジン,車体重量,加速性,$-馬力
0,21.9,86,2841,12.5,109.840827


* 参考
Scikit-learn で線形回帰 - Qiita 

https://qiita.com/0NE_shoT_/items/08376b08783cd554b02e

# 4. 予測モデルのスコアリングデータを作成（複数レコード）

In [7]:
import numpy as np
#fueleco=[21.9,22.0,22.1]
fueleco=np.arange(21.9, 22.1, 0.1)
engine=list(range(86,90))
weight=list(range(2841,2846))
#accel=[12.5,12.6,12.7,12.8]
accel=np.arange(12.5, 12.8, 0.1)
df_scoring2=pd.DataFrame(list(itertools.product(fueleco,engine,weight,accel)),
                         columns=['燃費', 'エンジン', '車体重量', '加速性'])

In [8]:
df_scoring2['$-馬力']=lr.predict(df_scoring2)
df_scoring2.sort_values(by=['$-馬力'],ascending=False)

,燃費,エンジン,車体重量,加速性,$-馬力
76,21.9,89,2845,12.5,110.098052
72,21.9,89,2844,12.5,110.076005
156,22.0,89,2845,12.5,110.054396
68,21.9,89,2843,12.5,110.053959
56,21.9,88,2845,12.5,110.041706
...,...,...,...,...,...
183,22.1,87,2841,12.8,108.349994
171,22.1,86,2843,12.8,108.337741
83,22.0,86,2841,12.8,108.337304
167,22.1,86,2842,12.8,108.315694


Python♪小数の繰り返し処理の落とし穴。range()、arange()、linspace() | Snow Tree in June 

https://snowtree-injune.com/2019/07/07/arange-linspace/

# 5. 各列のデータ型を指定したDataFrame生成

In [9]:
import datetime
df=pd.DataFrame({'INT': pd.Series([1], dtype=int),
                 'FLOAT': pd.Series([1], dtype=float),
                 'STR': pd.Series([1], dtype=str),
                 'DAY': [datetime.date(2000, 1, 31)],
                 'TS': [pd.to_datetime('2021-01-31 23:59:59')],
                 'TIME': ['13:20:58']
                })

df

,INT,FLOAT,STR,DAY,TS,TIME
0,1,1.0,1,2000-01-31,2021-01-31 23:59:59,13:20:58


In [10]:
print(df.dtypes)

INT               int32
FLOAT           float64
STR              object
DAY              object
TS       datetime64[ns]
TIME             object
dtype: object


In [11]:
for col in df.columns:
    print(type(df[col][0]))

<class 'numpy.int32'>
<class 'numpy.float64'>
<class 'str'>
<class 'datetime.date'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'str'>


In [12]:
import sys

print(sys.version)
print(pd.__version__)

3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]
1.4.1
